## 무신사 크롤링

In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By # 특정 선택자로 값을 찾아옴.
from selenium.webdriver.common.keys import Keys #명령어 넣을때
from webdriver_manager.chrome import ChromeDriverManager
import time

service = Service(ChromeDriverManager().install())
options = Options()
#options.add_argument('--headless') #headless를 쓰지 않으면 Grapic 소스가 불필요하게 충돌할 수 있음
#options.add_argument('--disable-gpu') #셀레니움 사용시, headless를 사용하지 않으면 브라우저가 켜짐. Gpu 실행. 이런 비효율성을 상대 페이지에서는 감지가 가능함. headless 사용시 사용하면 좋음
options.add_argument('--disable-dev-shm-usage') #로컬컴퓨터 메모리쓰기
options.add_argument('--window-size=1920x1080')
options.add_argument('--start-maximized') #반응형 페이지 방어
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36")
options.add_argument('--lang=ko_KR')

driver = webdriver.Chrome(service=service, options=options) #드라이버 실행

driver.get("https://www.musinsa.com/main/musinsa/recommend?gf=A")
time.sleep(3)

#검색버튼 누르게하기
search_button = driver.find_element(By.CLASS_NAME,"sc-dzagxs-2")
search_button.click()
time.sleep(2)

# 반팔버튼 누르기 - 순위가 
popular_keywords = driver.find_elements(By.CSS_SELECTOR,".search-home-popular-wrap .sc-1xzm8bl-1")
target_keyword = popular_keywords[2].text
print("목표 키워드 :", target_keyword)

#반팔 텍스트 검색어에 넣기
search_input = driver.find_element(By.CLASS_NAME,"search-home-search-bar-keyword")
search_input.clear()
search_input.send_keys(target_keyword)
time.sleep(1)
#엔터키를 자동으로 치게하기
search_input.send_keys(Keys.RETURN)
time.sleep(5)

print("검색완료. 현재 페이지 타이틀 : ", driver.title)

time.sleep(2)
driver.quit()

목표 키워드 : 반팔
검색완료. 현재 페이지 타이틀 :  반팔 | 무신사 추천 상품


## 무신사 크롤링 2 

In [33]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By # 특정 선택자로 값을 찾아옴.
from selenium.webdriver.common.keys import Keys #명령어 넣을때
from webdriver_manager.chrome import ChromeDriverManager
import time
import re # re : 어떤 값을 제거하는 remove 객체!

service = Service(ChromeDriverManager().install())
options = Options()
#options.add_argument('--headless') #headless를 쓰지 않으면 Grapic 소스가 불필요하게 충돌할 수 있음
#options.add_argument('--disable-gpu') #셀레니움 사용시, headless를 사용하지 않으면 브라우저가 켜짐. Gpu 실행. 이런 비효율성을 상대 페이지에서는 감지가 가능함. headless 사용시 사용하면 좋음
options.add_argument('--disable-dev-shm-usage') #로컬컴퓨터 메모리쓰기
options.add_argument('--window-size=1920x1080')
options.add_argument('--start-maximized') #반응형 페이지 방어
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36")
options.add_argument('--lang=ko_KR')

driver = webdriver.Chrome(service=service, options=options) #드라이버 실행

driver.get("https://www.musinsa.com/main/musinsa/recommend?gf=A")
time.sleep(3)

#검색버튼 누르게하기
search_button = driver.find_element(By.CLASS_NAME,"sc-dzagxs-2")
search_button.click()
time.sleep(2)

# 반팔버튼 누르기 - 순위가 
popular_keywords = driver.find_elements(By.CSS_SELECTOR,".search-home-popular-wrap .sc-1xzm8bl-1")
target_keyword = popular_keywords[2].text
print("목표 키워드 :", target_keyword)

#반팔 텍스트 검색어에 넣기
search_input = driver.find_element(By.CLASS_NAME,"search-home-search-bar-keyword")
search_input.clear()
search_input.send_keys(target_keyword)
time.sleep(1)
#엔터키를 자동으로 치게하기
search_input.send_keys(Keys.RETURN)
time.sleep(5)

print("검색완료. 현재 페이지 타이틀 : ", driver.title)

scroll_pause = 2
#10번반복하는 동안
for _ in range (10):
    #스크롤 밑으로 내림
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause) #반복할때마다 2초 쉼

products = driver.find_elements(By.CSS_SELECTOR,"div.sc-d36st-1.elXjMR")[:100] #계속찾아오는거 방지. 100개만 찾아와
product_data = []

for product in products:
    try :
        name = product.find_element(By.CSS_SELECTOR,"div.sc-cMuefe.jIlLeV a.gtm-select-item span").text.strip() #후손 선택자 : 한칸 띄우기
        url = product.find_element(By.CSS_SELECTOR,"div.sc-cMuefe.jIlLeV a.gtm-select-item").get_attribute("href") #속성값에 a 값인 url 넣어서 가져오기
        review_info = product.find_elements(By.CSS_SELECTOR,"span.text-etc_11px_reg.text-yellow.font-pretendard")[1].text.strip() #리뷰 평점 찾기
        review_count = int(re.sub(r"[^\d]","",review_info)) if review_info else 0  # review_info에 리뷰 값이 있으면 값을 주고 없으면 0을 준다.
        # r = 정규표현식을 쓰겠다. ^\d ^반대 \d 는 숫자! 숫자가 아닌 것을 ""(빈값)으로 sub(대체해라)
        product_data.append({"상품명": name, "URL": url, "리뷰수" : review_count})
    except :
        continue
        
print(product_data)

driver.quit()

목표 키워드 : 반팔
검색완료. 현재 페이지 타이틀 :  반팔 | 무신사 추천 브랜드
[{'상품명': 'Novichi T-Shirt Burgundy', 'URL': 'https://www.musinsa.com/products/4783409', '리뷰수': 78}, {'상품명': '후드 셔링 시스루 나시 레이어드 롱 슬림 반팔 티셔츠_ 2COLOR', 'URL': 'https://www.musinsa.com/products/5114155', '리뷰수': 3}, {'상품명': '3N605 Utility Poket T-Shirts (Wine)', 'URL': 'https://www.musinsa.com/products/4023372', '리뷰수': 8}, {'상품명': 'HBL OG 사이드테이프 오버핏 반팔 티셔츠 화이트(UP221CRS51)', 'URL': 'https://www.musinsa.com/products/4028590', '리뷰수': 33}, {'상품명': '마운틴 고트 스프레이 티셔츠 네이비', 'URL': 'https://www.musinsa.com/products/5025201', '리뷰수': 15}, {'상품명': '헤리티지 아치 로고 오버핏 반팔 티_5COLOR', 'URL': 'https://www.musinsa.com/products/4964420', '리뷰수': 28}, {'상품명': '베이직 저지 레오파드 릴렉스핏 반소매 티셔츠 - 레오파드 / T4124943', 'URL': 'https://www.musinsa.com/products/4791697', '리뷰수': 34}, {'상품명': '요트클럽 USA 티셔츠', 'URL': 'https://www.musinsa.com/products/4973206', '리뷰수': 42}, {'상품명': '에센셜 크롭 티셔츠 [차콜]', 'URL': 'https://www.musinsa.com/products/5170643', '리뷰수': 17}, {'상품명': 'BUTTON LAYERED CROP

## 판다스 쓰기

In [34]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By # 특정 선택자로 값을 찾아옴.
from selenium.webdriver.common.keys import Keys #명령어 넣을때
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd #판다스 가져오고 pd라는 이름으로 쓴다
import time
import re # re : 어떤 값을 제거하는 remove 객체!

service = Service(ChromeDriverManager().install())
options = Options()
#options.add_argument('--headless') #headless를 쓰지 않으면 Grapic 소스가 불필요하게 충돌할 수 있음
#options.add_argument('--disable-gpu') #셀레니움 사용시, headless를 사용하지 않으면 브라우저가 켜짐. Gpu 실행. 이런 비효율성을 상대 페이지에서는 감지가 가능함. headless 사용시 사용하면 좋음
options.add_argument('--disable-dev-shm-usage') #로컬컴퓨터 메모리쓰기
options.add_argument('--window-size=1920x1080')
options.add_argument('--start-maximized') #반응형 페이지 방어
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36")
options.add_argument('--lang=ko_KR')

driver = webdriver.Chrome(service=service, options=options) #드라이버 실행

driver.get("https://www.musinsa.com/main/musinsa/recommend?gf=A")
time.sleep(3)

#검색버튼 누르게하기
search_button = driver.find_element(By.CLASS_NAME,"sc-dzagxs-2")
search_button.click()
time.sleep(2)

# 반팔버튼 누르기 - 순위가 
popular_keywords = driver.find_elements(By.CSS_SELECTOR,".search-home-popular-wrap .sc-1xzm8bl-1")
target_keyword = popular_keywords[2].text
print("목표 키워드 :", target_keyword)

#반팔 텍스트 검색어에 넣기
search_input = driver.find_element(By.CLASS_NAME,"search-home-search-bar-keyword")
search_input.clear()
search_input.send_keys(target_keyword)
time.sleep(1)
#엔터키를 자동으로 치게하기
search_input.send_keys(Keys.RETURN)
time.sleep(5)

print("검색완료. 현재 페이지 타이틀 : ", driver.title)

scroll_pause = 2
#10번반복하는 동안
for _ in range (10):
    #스크롤 밑으로 내림
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause) #반복할때마다 2초 쉼

products = driver.find_elements(By.CSS_SELECTOR,"div.sc-d36st-1.elXjMR")[:100] #계속찾아오는거 방지. 100개만 찾아와
product_data = []

for product in products:
    try :
        name = product.find_element(By.CSS_SELECTOR,"div.sc-cMuefe.jIlLeV a.gtm-select-item span").text.strip() #후손 선택자 : 한칸 띄우기
        url = product.find_element(By.CSS_SELECTOR,"div.sc-cMuefe.jIlLeV a.gtm-select-item").get_attribute("href") #속성값에 a 값인 url 넣어서 가져오기
        review_info = product.find_elements(By.CSS_SELECTOR,"span.text-etc_11px_reg.text-yellow.font-pretendard")[1].text.strip() #리뷰 평점 찾기
        review_count = int(re.sub(r"[^\d]","",review_info)) if review_info else 0  # review_info에 리뷰 값이 있으면 값을 주고 없으면 0을 준다.
        # r = 정규표현식을 쓰겠다. ^\d ^반대 \d 는 숫자! 숫자가 아닌 것을 ""(빈값)으로 sub(대체해라)
        product_data.append({"상품명": name, "URL": url, "리뷰수" : review_count})
    except :
        continue
# 판다스!!!!!!!!###############################################
df = pd.DataFrame(product_data)
df_top30 = df.sort_values(by="리뷰수", ascending=False).head(30).reset_index(drop=True) 
#리뷰수를 기준으로 sorting을 시킴, ascending 시키지마라 = 내림차순, 30개 잘라라, 나머지는 버려라

driver.quit()
df_top30.head()

목표 키워드 : 반팔
검색완료. 현재 페이지 타이틀 :  반팔 | 무신사 추천 브랜드


,상품명,URL,리뷰수
0,Rowdy Basketball League T-Shirt_Oatmeal,https://www.musinsa.com/products/4147228,724
1,UNIVERSITY PIGMENT TEE CHARCOAL(MG2CMMT531A),https://www.musinsa.com/products/2404968,712
2,원캣 우먼 레귤러 베이직 티셔츠,https://www.musinsa.com/products/3401498,705
3,[2PACK]돌핀 아치로고 티셔츠 화이트 블랙 2팩,https://www.musinsa.com/products/3060324,585
4,[쿨탠다드] LV2 액티브 숏 슬리브 (3color),https://www.musinsa.com/products/3535315,425


## 진짜 리뷰(텍스트)들을 추출하는 방법 

In [44]:
from openpyxl import Workbook

In [ ]:

######################리뷰 텍스트들 추출 ###################################
# 1, 위에 openpyxl!
driver = webdriver.Chrome(service=service, options=options)
all_reviews = []

for idx, row in df_top30.iterrows(): #iterrows : 1개의 행을 가져오고 반복할때마다 1개의 값을 찾아오게 하는 함수
    print(f"{idx + 1}번째 상품리뷰 수집 중 :", {row['상품명']})
    driver.get(row["URL"]) #row라는 행 안에 URL이라는 값에 매칭되는 애를 찾아오겠다
    time.sleep(2)

#오류 발생할 수도 있으니까 try
    try :
        review_tab = driver.find_element(By.XPATH,"//button[@data-button-id='prd_review_tab']") # id 안전빵으로 찾아오고 xpath
        review_tab.click()
        time.sleep(2)
        #모든리뷰 X 한 상품당 10개정도만 스크롤해서 찾아오기
        for _ in range(3) :
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
            time.sleep(1.5)

        # 스크롤할때마다 리뷰 찾아오기
        review_elements = driver.find_elements(By.CSS_SELECTOR,"span.text-body_13px_reg.text-black.font-pretendard")
        for review in review_elements[:10] :
            text = review.text.strip()
            if text : 
                all_reviews.append({
                    "상품명" : row["상품명"], #row상품명에서 가져옴
                    "리뷰" : text
                }) # 비어있는 리스트에 값추가

    except Exception as e:
        print("리뷰 수집 실패 : ", e)
        continue

driver.quit()
#엑셀 저장
wb = Workbook()
ws = wb.active
ws.title = "무신사 리뷰"

ws. append(["상품명","리뷰"])

#각각의 리뷰안에 상품명 한쌍으로 만들기
for r in all_reviews:
    ws.append([r["상품명"],r["리뷰"]])

wb.save("musinsa_top30_reviews.xlsx")
print("⭐엑셀 저장완료 : musinsa_top30_reviews.xlsx")

1번째 상품리뷰 수집 중 : {'Rowdy Basketball League T-Shirt_Oatmeal'}
리뷰 수집 실패 :  Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[@data-button-id='prd_review_tab']"}
  (Session info: chrome=138.0.7204.184); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
	GetHandleVerifier [0x0x123ba83+63395]
	GetHandleVerifier [0x0x123bac4+63460]
	(No symbol) [0x0x1082113]
	(No symbol) [0x0x10ca85e]
	(No symbol) [0x0x10cabfb]
	(No symbol) [0x0x1112f92]
	(No symbol) [0x0x10ef3f4]
	(No symbol) [0x0x11107ba]
	(No symbol) [0x0x10ef1a6]
	(No symbol) [0x0x10be7b2]
	(No symbol) [0x0x10bf654]
	GetHandleVerifier [0x0x14b8883+2672035]
	GetHandleVerifier [0x0x14b3cba+2652634]
	GetHandleVerifier [0x0x1262bca+223466]
	GetHandleVerifier [0x0x1252cb8+158168]
	GetHandleVerifier [0x0x125978d+185517]
	GetHandleVerifier [0x0x1243b78+96408]
	GetHandleVerifier [0x0x1243d02+96802]
	GetH

In [43]:
pip install sqlalchemy

In [52]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine # sql테이블로 바로 보냄

EXCEL_PATH = "musinsa_top30_reviews.xlsx"
DB_NAME = "musinsa_db"
TABLE_NAME = "reviews"
USER = "root"
PASSWORD = "asdf1122"
HOST = "127.0.0.1"
PORT = 3306
df = pd.read_excel(EXCEL_PATH)

conn = pymysql.connect(host=HOST, user = USER, password = PASSWORD, port = PORT)
cursor = conn.cursor()

cursor.execute(
f"CREATE DATABASE IF NOT EXISTS {DB_NAME} CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;"
)
conn.close()

engine_str = f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DB_NAME}?charset=utf8mb4"
engine = create_engine(engine_str)

df.to_sql(name=TABLE_NAME, con=engine, if_exists="replace", index=False)

print(f"⭐데이터가 MYSQL[{DB_NAME}]의 테이블 [{TABLE_NAME}]에 성공적으로 저장되었습니다.")

⭐데이터가 MYSQL[musinsa_db]의 테이블 [reviews]에 성공적으로 저장되었습니다.
